In [1]:
import deep_main
import pandas as pd
import os

In [2]:


ntp = 1 # Number of trees for each path printed to result directory, -1 implies printing all of them
plot_predir = './result' # string: path of result saved
binary_out = True # boolean: True if outcome is binary 
analyze_method = 'xgb' # defining tree method
outcome_names = ['DailyControllerMedication', 'EmergencyRoomVisit']
confounding_vars = ['age', 'avg_income',
                    'race/ethnicity_Asian',
                    'race/ethnicity_Black or African American',
                    'race/ethnicity_Hispanic or Latino',
                    'race/ethnicity_More Than One Race',
                    'race/ethnicity_Unknown / Not Reported',
                    'race/ethnicity_White',
                    'gender',
                    ]

irrelevant_vars = ["ID"]




# Output files

# file = args.filename

In [3]:
if 'xgb' in analyze_method:
    xgb_predict = True
else:
    xgb_predict = False

if not os.path.exists(plot_predir):
    os.mkdir(plot_predir)

pvalue_df_list = []
pred_score_df_list = []

plotDir = plot_predir

for outcome in outcome_names:
    filename = outcome+'.csv'
    pvalue_df, pred_score_df = deep_main.runWorkflow(input_file=filename,
                                                    binary_outcome=binary_out,
                                                    output_folder_name=outcome,
                                                    confounding_vars=confounding_vars,
                                                    irrelevant_vars=irrelevant_vars,
                                                    xgb=xgb_predict,
                                                    num_tree_print=ntp
                                                    )
    pvalue_df.dropna(axis=0, how='any', inplace=True)
    pvalue_df.to_csv(os.path.join(plot_predir, 'pval_{}_{}.csv'.format(outcome, analyze_method)), index=False, header=True)
    pred_score_df.to_csv(os.path.join(plot_predir, 'pred_score_{}_{}.csv'.format(outcome, analyze_method)), index=False, header=True)



... class distribution | classes: [1, 0] | sizes: Counter({1: 87, 0: 62})
... dim(X): (149, 125) variables:
Start running stage 1 ...

0th run result finished!
> 1. Frequent decision paths overall ...
0th paths (5):	FORMALDEHYDE<1.652e+00
1th paths (5):	ETHYL ACRYLATE<4.195e-06
2th paths (5):	METHYL METHACRYLATE>=3.044e-03
3th paths (5):	CHLOROACETIC ACID<3.130e-07		ETHYLENE OXIDE>=2.911e-04
4th paths (4):	ACETOPHENONE<1.470e-05
5th paths (4):	POLYCHLORINATED BIPHENYLS (AROCLORS)<1.185e-08
6th paths (4):	PHOSPHORUS<1.107e-04
7th paths (4):	HYDROQUINONE<7.255e-06		1,4-DIOXANE>=1.605e-06
8th paths (4):	ARSENIC COMPOUNDS(INORGANIC INCLUDING ARSINE)<5.820e-04
9th paths (3):	LEAD COMPOUNDS<1.462e-03
Finished All regressions!
... class distribution | classes: [1, 0] | sizes: Counter({1: 110, 0: 41})
... dim(X): (151, 125) variables:
Start running stage 1 ...

0th run result finished!
> 1. Frequent decision paths overall ...
0th paths (5):	DIMETHYL SULFATE<6.996e-07
1th paths (4):	CHLOROPRENE

In [4]:
import sys
import os
import pandas as pd
from statsmodels.stats.multitest import fdrcorrection
import argparse
import fnmatch

def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            # print(name)
            if fnmatch.fnmatch(name, pattern):
                # print('matched')
                result.append(name)
    return result

result_dir = plot_predir

fdr_file_list = find('pval_*.csv', result_dir)

p_val_list = []
pred_score_list = []
for fdr_file in fdr_file_list:
    p_val_list.append(pd.read_csv(os.path.join(result_dir,fdr_file)))

pred_file_list = find('pred_score*.csv', result_dir)
for pred_file in pred_file_list:
    pred_score_list.append(pd.read_csv(os.path.join(result_dir,pred_file)))

pvalue_df_cat = pd.concat(p_val_list, ignore_index=True)
pred_score_df_cat = pd.concat(pred_score_list, ignore_index=True)

pvalue_df_cat['fdr'] = 0.0
bool_list_fdr = [
                    (pvalue_df_cat['binary_outcome'] == True),
                    (pvalue_df_cat['binary_outcome'] == False)
                    ]
for b in bool_list_fdr:
    pvalue_df_cat.loc[b,'fdr'] = fdrcorrection(pvalue_df_cat.loc[b,'p_val'].values)[-1]

pvalue_df_cat.to_csv(os.path.join(result_dir, 'merged_fdr.csv'), index=False, header=True)
pred_score_df_cat.to_csv(os.path.join(result_dir,'merged_pred_score.csv'), index=False, header=True)